In [3]:
def get_datasets(
    dataset: pd.DataFrame,
    scaler: TransformerMixin,
    validation_size: float = 0.2,
    test_size: float = 0.1,
    batch_size: int = 32,
    random_state: int = 42,
) -> Tuple[tf.data.Dataset]:
    """Converts a whole dataset into train, validation and test sets
       as tf.data.Dataset pipelines
    Args:
        dataset (pd.DataFrame): a dataframe containing X and y data
        scaler (TransformerMixin): a transformer performing scaling
        i.e StandardScaler()
        validation_size (float, optional): the size of the validation
        set (in percent). Defaults to 0.2.
        test_size (float, optional): the size of the test set
        (in percent). Defaults to 0.1.
        batch_size (int, optional): the size of the batch during
        training. Defaults to 32.
    Returns:
        Tuple[tf.data.Dataset]: a tuple containing train, validation
        and test set pipelines
    """
    X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )

    X_train, X_val, y_train, y_val = train_test_split(
        X_train,
        y_train,
        train_size=1 - validation_size / (1 - test_size),
        stratify=y_train,
        random_state=random_state,
    )

    # all features are actually the same feature from different
    # timestamps

    X_train_flattened = X_train.values.reshape(-1, 1)
    scaler.fit(X_train_flattened)

    for col in X_train.columns:
        X_train[[col]] = scaler.transform(X_train[[col]])
        X_val[[col]] = scaler.transform(X_val[[col]])
        X_test[[col]] = scaler.transform(X_test[[col]])

    train_set = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .batch(batch_size)
        .prefetch(3)
    )

    val_set = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val))
        .batch(batch_size)
        .prefetch(3)
    )

    test_set = (
        tf.data.Dataset.from_tensor_slices((X_test, y_test))
        .batch(batch_size)
        .prefetch(3)
    )

    return (train_set, val_set, test_set)

In [13]:
from typing import Tuple

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.base import TransformerMixin
from sklearn.model_selection import train_test_split

In [4]:
import tensorflow.keras.backend as K


def custom_sigmoid(x):
    return 2 * K.sigmoid(37 * x) - 1

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))

In [147]:
dataset_dir = {
    7: 'dataset_freq_7.csv',
    8: 'dataset_freq_8.csv',
    9: 'dataset_freq_9.csv',
    10: 'dataset_freq_10.csv',
    11: 'dataset_freq_11.csv',
    12: 'dataset_freq_12.csv',
    13: 'dataset_freq_13.csv'
}

In [301]:
histories = []
for freq, csv_name in dataset_dir.items():
    opt = tf.keras.optimizers.SGD(learning_rate=0.005,
    momentum=0.9,
    nesterov=True)
    loss_fn = tf.keras.losses.BinaryCrossentropy()
    
    train_set, val_set, test_set = get_datasets(pd.read_csv(csv_name), scaler)
    
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(freq,)),
        tf.keras.layers.Dense(freq, use_bias=False),
        tf.keras.layers.Activation(custom_sigmoid, name='custom_sigmoid'),
        tf.keras.layers.Dense(1, activation='sigmoid', use_bias=False)
    ])
    model.compile(optimizer=opt,
              loss=loss_fn,
              metrics=['accuracy'])
    history = model.fit(train_set, epochs=30,
                    validation_data=val_set)
    histories.append(history)

Epoch 1/30


C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\k

133/133 [==============================] - 1s 4ms/step - loss: 0.6602 - accuracy: 0.6165 - val_loss: 0.6392 - val_accuracy: 0.6311
Epoch 2/30
133/133 [==============================] - 0s 3ms/step - loss: 0.6049 - accuracy: 0.6574 - val_loss: 0.5607 - val_accuracy: 0.7064
Epoch 3/30
133/133 [==============================] - 0s 3ms/step - loss: 0.5206 - accuracy: 0.7643 - val_loss: 0.4847 - val_accuracy: 0.8015
Epoch 4/30
133/133 [==============================] - 0s 3ms/step - loss: 0.4488 - accuracy: 0.8225 - val_loss: 0.4249 - val_accuracy: 0.8503
Epoch 5/30
133/133 [==============================] - 0s 3ms/step - loss: 0.3969 - accuracy: 0.8603 - val_loss: 0.3842 - val_accuracy: 0.8710
Epoch 6/30
133/133 [==============================] - 0s 2ms/step - loss: 0.3599 - accuracy: 0.8792 - val_loss: 0.3550 - val_accuracy: 0.8867
Epoch 7/30
133/133 [==============================] - 0s 3ms/step - loss: 0.3351 - accuracy: 0.8875 - val_loss: 0.3374 - val_accuracy: 0.8933
Epoch 8/30
133/13

C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\k

133/133 [==============================] - 1s 3ms/step - loss: 0.6480 - accuracy: 0.6797 - val_loss: 0.5557 - val_accuracy: 0.7494
Epoch 2/30
133/133 [==============================] - 0s 2ms/step - loss: 0.5203 - accuracy: 0.7846 - val_loss: 0.4963 - val_accuracy: 0.7833
Epoch 3/30
133/133 [==============================] - 0s 2ms/step - loss: 0.4617 - accuracy: 0.8121 - val_loss: 0.4441 - val_accuracy: 0.7949
Epoch 4/30
133/133 [==============================] - 0s 2ms/step - loss: 0.4160 - accuracy: 0.8210 - val_loss: 0.4036 - val_accuracy: 0.8015
Epoch 5/30
133/133 [==============================] - 0s 2ms/step - loss: 0.3809 - accuracy: 0.8376 - val_loss: 0.3673 - val_accuracy: 0.8354
Epoch 6/30
133/133 [==============================] - 0s 2ms/step - loss: 0.3466 - accuracy: 0.8603 - val_loss: 0.3278 - val_accuracy: 0.8685
Epoch 7/30
133/133 [==============================] - 0s 2ms/step - loss: 0.3149 - accuracy: 0.8809 - val_loss: 0.2957 - val_accuracy: 0.8958
Epoch 8/30
133/13

...

133/133 [==============================] - 1s 3ms/step - loss: 0.6103 - accuracy: 0.6846 - val_loss: 0.4941 - val_accuracy: 0.7601
Epoch 2/30
133/133 [==============================] - 0s 2ms/step - loss: 0.4629 - accuracy: 0.7983 - val_loss: 0.4163 - val_accuracy: 0.8371
Epoch 3/30
133/133 [==============================] - 0s 2ms/step - loss: 0.3972 - accuracy: 0.8610 - val_loss: 0.3636 - val_accuracy: 0.8685
Epoch 4/30
133/133 [==============================] - 0s 2ms/step - loss: 0.3459 - accuracy: 0.8882 - val_loss: 0.3138 - val_accuracy: 0.8925
Epoch 5/30
133/133 [==============================] - 0s 2ms/step - loss: 0.2998 - accuracy: 0.9106 - val_loss: 0.2769 - val_accuracy: 0.9140
Epoch 6/30
133/133 [==============================] - 0s 2ms/step - loss: 0.2686 - accuracy: 0.9191 - val_loss: 0.2532 - val_accuracy: 0.9214
Epoch 7/30
133/133 [==============================] - 0s 2ms/step - loss: 0.2464 - accuracy: 0.9265 - val_loss: 0.2364 - val_accuracy: 0.9289
Epoch 8/30
133/13

C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\k

133/133 [==============================] - 1s 3ms/step - loss: 0.6286 - accuracy: 0.6697 - val_loss: 0.5000 - val_accuracy: 0.7907
Epoch 2/30
133/133 [==============================] - 0s 2ms/step - loss: 0.4639 - accuracy: 0.8106 - val_loss: 0.4146 - val_accuracy: 0.8412
Epoch 3/30
133/133 [==============================] - 0s 2ms/step - loss: 0.4058 - accuracy: 0.8409 - val_loss: 0.3681 - val_accuracy: 0.8569
Epoch 4/30
133/133 [==============================] - 0s 2ms/step - loss: 0.3636 - accuracy: 0.8655 - val_loss: 0.3260 - val_accuracy: 0.8908
Epoch 5/30
133/133 [==============================] - 0s 2ms/step - loss: 0.3220 - accuracy: 0.8998 - val_loss: 0.2887 - val_accuracy: 0.9123
Epoch 6/30
133/133 [==============================] - 0s 2ms/step - loss: 0.2905 - accuracy: 0.9154 - val_loss: 0.2633 - val_accuracy: 0.9156
Epoch 7/30
133/133 [==============================] - 0s 2ms/step - loss: 0.2673 - accuracy: 0.9229 - val_loss: 0.2443 - val_accuracy: 0.9222
Epoch 8/30
133/13

C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\k

133/133 [==============================] - 1s 4ms/step - loss: 0.6433 - accuracy: 0.6806 - val_loss: 0.5612 - val_accuracy: 0.7477
Epoch 2/30
133/133 [==============================] - 0s 3ms/step - loss: 0.5240 - accuracy: 0.7641 - val_loss: 0.4553 - val_accuracy: 0.7907
Epoch 3/30
133/133 [==============================] - 0s 3ms/step - loss: 0.4341 - accuracy: 0.8087 - val_loss: 0.3884 - val_accuracy: 0.8296
Epoch 4/30
133/133 [==============================] - 0s 2ms/step - loss: 0.3759 - accuracy: 0.8456 - val_loss: 0.3386 - val_accuracy: 0.8734
Epoch 5/30
133/133 [==============================] - 0s 3ms/step - loss: 0.3280 - accuracy: 0.8861 - val_loss: 0.2962 - val_accuracy: 0.9074
Epoch 6/30
133/133 [==============================] - 0s 2ms/step - loss: 0.2890 - accuracy: 0.9064 - val_loss: 0.2653 - val_accuracy: 0.9198
Epoch 7/30
133/133 [==============================] - 0s 2ms/step - loss: 0.2619 - accuracy: 0.9173 - val_loss: 0.2438 - val_accuracy: 0.9239
Epoch 8/30
133/13

In [302]:
max_val_acc = [history.history['val_accuracy'][-1] for history in histories]

In [303]:
max_val_acc

[0.9181141257286072,
 0.9387924075126648,
 0.9454094171524048,
 0.947063684463501,
 0.9445822834968567,
 0.947063684463501,
 0.9561620950698853]

Największą wartość ma input 13 (czego można się było domyślić), ale decydujemy się na 10, bo jest tańsza sprzętowo.

In [6]:
train_set, val_set, test_set = get_datasets(pd.read_csv('dataset_freq_10.csv'), scaler)

C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\kicpe\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
C:\Users\k

# ==== DLA NATALII ===

In [219]:
dt = np.dtype(np.int32)  
arr = np.array([1,2,3])
arr_ = np.array([2,3,4])

ls = []
# arr = np.append(arr, [[1,2]], axis=1)
# arr = np.append(arr, [[2,3]], axis=0)
# arr = np.append(arr, [[2,3]], axis=0)
# print(arr)
for s in test_set:
    for idx, row in enumerate(s[0]):
        single_row = np.concatenate((np.array(row), np.array(s[1][idx])), axis=None)
        ls.append(single_row)
        
ls = np.array(ls)
print(ls.shape)

pd.DataFrame(ls).to_csv("dla_natalii.csv", header=False, index=False)


(605, 11)


# ================

In [31]:
%tensorboard --port 6005 --logdir logs/fit

In [7]:
l2_regularizers = [
    tf.keras.regularizers.L2(l2=0.008),
    tf.keras.regularizers.L2(l2=0.007),
    tf.keras.regularizers.L2(l2=0.006)
]

init_kernels = [
    'glorot_normal',
    'he_normal'
]

In [32]:
def build_model(l2_regularizer, init_kernel):
    metrics = [
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.BinaryAccuracy(name='accuracy')
    ]
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(10,)),
        tf.keras.layers.Dense(10, use_bias=False, kernel_initializer=init_kernel, 
                            kernel_regularizer=l2_regularizer),
        tf.keras.layers.Activation(custom_sigmoid, name='custom_sigmoid'),
        tf.keras.layers.Dense(1, activation='sigmoid', use_bias=False, kernel_initializer=init_kernel,
                                  kernel_regularizer=l2_regularizer)
    ])
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=metrics)
    return model

def train_model(model, l2_regularizer, init_kernel):
    log_dir = "logs/fit/" + str(l2_regularizer.l2) + '_' + init_kernel
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, 
                                                          histogram_freq=1,
                                                          write_images=True)
    early_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=1e-3)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
    
    history = model.fit(
        train_set, 
        epochs=250, 
        validation_data=val_set, 
        callbacks=[tensorboard_callback, early_callback, reduce_lr]) 
    
    return model, history
    
def evaluate_model(model):
    metrics = [
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.BinaryAccuracy(name='accuracy')
    ]
    model.layers[-1].activate = custom_sigmoid
    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True), 
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=metrics)
    res = model.evaluate(test_set)
    return res, model

def process_models(l2_regularizers, init_kernels):
    results, models, histories = [], [], []
    for init_kernel in init_kernels:
        for l2_regularizer in l2_regularizers:
            model = build_model(l2_regularizer, init_kernel)
            model, history = train_model(model, l2_regularizer, init_kernel)
            res, model = evaluate_model(model)
            results.append(res)
            models.append(model)
            histories.append(history)
    return results, models, histories

In [33]:
results, models, histories = process_models(l2_regularizers, init_kernels)

Epoch 1/250
133/133 [==============================] - 2s 10ms/step - loss: 0.6372 - auc: 0.7789 - recall: 0.4700 - precision: 0.7739 - accuracy: 0.7449 - val_loss: 0.5276 - val_auc: 0.8857 - val_recall: 0.6212 - val_precision: 0.8645 - val_accuracy: 0.8180 - lr: 0.0100
Epoch 2/250
133/133 [==============================] - 1s 5ms/step - loss: 0.4967 - auc: 0.9114 - recall: 0.6759 - precision: 0.8879 - accuracy: 0.8435 - val_loss: 0.4688 - val_auc: 0.9342 - val_recall: 0.7532 - val_precision: 0.8810 - val_accuracy: 0.8668 - lr: 0.0100
Epoch 3/250
133/133 [==============================] - 1s 6ms/step - loss: 0.4478 - auc: 0.9463 - recall: 0.7755 - precision: 0.9214 - accuracy: 0.8889 - val_loss: 0.4357 - val_auc: 0.9551 - val_recall: 0.8723 - val_precision: 0.8818 - val_accuracy: 0.9065 - lr: 0.0100
Epoch 4/250
133/133 [==============================] - 1s 5ms/step - loss: 0.4224 - auc: 0.9586 - recall: 0.8312 - precision: 0.9250 - accuracy: 0.9097 - val_loss: 0.4156 - val_auc: 0.9644 

Epoch 4/250
133/133 [==============================] - 1s 6ms/step - loss: 0.4113 - auc: 0.9589 - recall: 0.8275 - precision: 0.9416 - accuracy: 0.9144 - val_loss: 0.4184 - val_auc: 0.9584 - val_recall: 0.8810 - val_precision: 0.8965 - val_accuracy: 0.9156 - lr: 0.0100
Epoch 5/250
133/133 [==============================] - 1s 6ms/step - loss: 0.4033 - auc: 0.9619 - recall: 0.8466 - precision: 0.9435 - accuracy: 0.9220 - val_loss: 0.4131 - val_auc: 0.9606 - val_recall: 0.8896 - val_precision: 0.9033 - val_accuracy: 0.9214 - lr: 0.0100
Epoch 6/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3995 - auc: 0.9635 - recall: 0.8479 - precision: 0.9442 - accuracy: 0.9227 - val_loss: 0.4104 - val_auc: 0.9615 - val_recall: 0.8874 - val_precision: 0.9051 - val_accuracy: 0.9214 - lr: 0.0100
Epoch 7/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3977 - auc: 0.9644 - recall: 0.8491 - precision: 0.9469 - accuracy: 0.9241 - val_loss: 0.4087 - val_auc: 0.9623 -

133/133 [==============================] - 1s 5ms/step - loss: 0.3765 - auc: 0.9717 - recall: 0.8695 - precision: 0.9474 - accuracy: 0.9317 - val_loss: 0.3831 - val_auc: 0.9721 - val_recall: 0.8961 - val_precision: 0.9119 - val_accuracy: 0.9272 - lr: 0.0100
Epoch 35/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3764 - auc: 0.9717 - recall: 0.8689 - precision: 0.9468 - accuracy: 0.9312 - val_loss: 0.3829 - val_auc: 0.9721 - val_recall: 0.8961 - val_precision: 0.9119 - val_accuracy: 0.9272 - lr: 0.0100
Epoch 36/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3764 - auc: 0.9717 - recall: 0.8695 - precision: 0.9468 - accuracy: 0.9314 - val_loss: 0.3827 - val_auc: 0.9721 - val_recall: 0.8961 - val_precision: 0.9139 - val_accuracy: 0.9280 - lr: 0.0100
Epoch 37/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3763 - auc: 0.9717 - recall: 0.8695 - precision: 0.9468 - accuracy: 0.9314 - val_loss: 0.3826 - val_auc: 0.9721 - val_reca

Epoch 18/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3547 - auc: 0.9788 - recall: 0.8850 - precision: 0.9585 - accuracy: 0.9414 - val_loss: 0.3700 - val_auc: 0.9749 - val_recall: 0.9134 - val_precision: 0.9295 - val_accuracy: 0.9404 - lr: 0.0100
Epoch 19/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3546 - auc: 0.9788 - recall: 0.8850 - precision: 0.9585 - accuracy: 0.9414 - val_loss: 0.3701 - val_auc: 0.9748 - val_recall: 0.9134 - val_precision: 0.9295 - val_accuracy: 0.9404 - lr: 0.0100
Epoch 20/250
133/133 [==============================] - 1s 4ms/step - loss: 0.3517 - auc: 0.9805 - recall: 0.8831 - precision: 0.9629 - accuracy: 0.9423 - val_loss: 0.3627 - val_auc: 0.9755 - val_recall: 0.8831 - val_precision: 0.9423 - val_accuracy: 0.9347 - lr: 0.0020
Epoch 21/250
133/133 [==============================] - 1s 4ms/step - loss: 0.3503 - auc: 0.9806 - recall: 0.8868 - precision: 0.9624 - accuracy: 0.9435 - val_loss: 0.3627 - val_auc: 0.97

Epoch 18/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3964 - auc: 0.9753 - recall: 0.8683 - precision: 0.9623 - accuracy: 0.9366 - val_loss: 0.4041 - val_auc: 0.9735 - val_recall: 0.9004 - val_precision: 0.9369 - val_accuracy: 0.9388 - lr: 0.0100
Epoch 19/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3959 - auc: 0.9754 - recall: 0.8695 - precision: 0.9624 - accuracy: 0.9371 - val_loss: 0.4036 - val_auc: 0.9737 - val_recall: 0.8983 - val_precision: 0.9368 - val_accuracy: 0.9380 - lr: 0.0100
Epoch 20/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3955 - auc: 0.9755 - recall: 0.8701 - precision: 0.9624 - accuracy: 0.9374 - val_loss: 0.4032 - val_auc: 0.9737 - val_recall: 0.8983 - val_precision: 0.9368 - val_accuracy: 0.9380 - lr: 0.0100
Epoch 21/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3951 - auc: 0.9755 - recall: 0.8701 - precision: 0.9630 - accuracy: 0.9376 - val_loss: 0.4028 - val_auc: 0.97

Epoch 13/250
133/133 [==============================] - 1s 4ms/step - loss: 0.3862 - auc: 0.9705 - recall: 0.8596 - precision: 0.9508 - accuracy: 0.9293 - val_loss: 0.3913 - val_auc: 0.9709 - val_recall: 0.8983 - val_precision: 0.9181 - val_accuracy: 0.9305 - lr: 0.0100
Epoch 14/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3844 - auc: 0.9714 - recall: 0.8602 - precision: 0.9534 - accuracy: 0.9305 - val_loss: 0.3889 - val_auc: 0.9716 - val_recall: 0.8983 - val_precision: 0.9222 - val_accuracy: 0.9322 - lr: 0.0100
Epoch 15/250
133/133 [==============================] - 1s 4ms/step - loss: 0.3825 - auc: 0.9723 - recall: 0.8615 - precision: 0.9541 - accuracy: 0.9312 - val_loss: 0.3865 - val_auc: 0.9724 - val_recall: 0.9004 - val_precision: 0.9224 - val_accuracy: 0.9330 - lr: 0.0100
Epoch 16/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3807 - auc: 0.9731 - recall: 0.8646 - precision: 0.9556 - accuracy: 0.9329 - val_loss: 0.3846 - val_auc: 0.97

133/133 [==============================] - 1s 5ms/step - loss: 0.3724 - auc: 0.9771 - recall: 0.8732 - precision: 0.9579 - accuracy: 0.9369 - val_loss: 0.3753 - val_auc: 0.9762 - val_recall: 0.9048 - val_precision: 0.9351 - val_accuracy: 0.9396 - lr: 0.0100
Epoch 44/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3692 - auc: 0.9787 - recall: 0.8782 - precision: 0.9608 - accuracy: 0.9397 - val_loss: 0.3704 - val_auc: 0.9772 - val_recall: 0.8961 - val_precision: 0.9474 - val_accuracy: 0.9413 - lr: 0.0020
Epoch 45/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3680 - auc: 0.9788 - recall: 0.8806 - precision: 0.9615 - accuracy: 0.9409 - val_loss: 0.3703 - val_auc: 0.9771 - val_recall: 0.8961 - val_precision: 0.9452 - val_accuracy: 0.9404 - lr: 0.0020
Epoch 46/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3679 - auc: 0.9787 - recall: 0.8806 - precision: 0.9622 - accuracy: 0.9411 - val_loss: 0.3703 - val_auc: 0.9770 - val_reca

Epoch 19/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3605 - auc: 0.9759 - recall: 0.8751 - precision: 0.9554 - accuracy: 0.9366 - val_loss: 0.3675 - val_auc: 0.9758 - val_recall: 0.9134 - val_precision: 0.9174 - val_accuracy: 0.9355 - lr: 0.0100
Epoch 20/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3604 - auc: 0.9759 - recall: 0.8751 - precision: 0.9561 - accuracy: 0.9369 - val_loss: 0.3674 - val_auc: 0.9759 - val_recall: 0.9134 - val_precision: 0.9154 - val_accuracy: 0.9347 - lr: 0.0100
Epoch 21/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3602 - auc: 0.9760 - recall: 0.8751 - precision: 0.9561 - accuracy: 0.9369 - val_loss: 0.3673 - val_auc: 0.9759 - val_recall: 0.9134 - val_precision: 0.9154 - val_accuracy: 0.9347 - lr: 0.0100
Epoch 22/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3601 - auc: 0.9761 - recall: 0.8745 - precision: 0.9554 - accuracy: 0.9364 - val_loss: 0.3672 - val_auc: 0.97

133/133 [==============================] - 1s 5ms/step - loss: 0.3573 - auc: 0.9761 - recall: 0.8732 - precision: 0.9579 - accuracy: 0.9369 - val_loss: 0.3636 - val_auc: 0.9761 - val_recall: 0.9113 - val_precision: 0.9192 - val_accuracy: 0.9355 - lr: 0.0100
Epoch 50/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3572 - auc: 0.9761 - recall: 0.8732 - precision: 0.9579 - accuracy: 0.9369 - val_loss: 0.3634 - val_auc: 0.9761 - val_recall: 0.9113 - val_precision: 0.9192 - val_accuracy: 0.9355 - lr: 0.0100
Epoch 51/250
133/133 [==============================] - 1s 4ms/step - loss: 0.3572 - auc: 0.9762 - recall: 0.8732 - precision: 0.9579 - accuracy: 0.9369 - val_loss: 0.3633 - val_auc: 0.9762 - val_recall: 0.9134 - val_precision: 0.9174 - val_accuracy: 0.9355 - lr: 0.0100
Epoch 52/250
133/133 [==============================] - 1s 5ms/step - loss: 0.3571 - auc: 0.9762 - recall: 0.8745 - precision: 0.9580 - accuracy: 0.9374 - val_loss: 0.3632 - val_auc: 0.9762 - val_reca

In [359]:
models[1].history.history.keys()

dict_keys([])

In [24]:
histories[0].history['binary_accuracy']

[0.8156028389930725,
 0.8810874819755554,
 0.895035445690155,
 0.9026004672050476,
 0.9066193699836731,
 0.9120567440986633,
 0.9144207835197449,
 0.9177305102348328,
 0.9196217656135559,
 0.9208037853240967,
 0.922931432723999,
 0.9236406683921814,
 0.9243499040603638,
 0.9257683157920837,
 0.9271867871284485,
 0.9283688068389893,
 0.9286051988601685,
 0.9297872185707092,
 0.9302600622177124,
 0.9307328462600708,
 0.931205689907074,
 0.9314420819282532,
 0.9316784739494324,
 0.9319148659706116,
 0.9319148659706116,
 0.9319148659706116,
 0.9321513175964355,
 0.9316784739494324,
 0.9321513175964355,
 0.9323877096176147,
 0.9319148659706116,
 0.9319148659706116,
 0.9316784739494324,
 0.9316784739494324,
 0.9319148659706116,
 0.9319148659706116,
 0.9323877096176147,
 0.932624101638794,
 0.9323877096176147,
 0.9321513175964355,
 0.9316784739494324,
 0.9316784739494324,
 0.9314420819282532,
 0.9314420819282532,
 0.931205689907074,
 0.9314420819282532,
 0.9314420819282532,
 0.931442081928253

In [330]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 5004), started 2 days, 20:54:08 ago. (Use '!kill 5004' to kill it.)

# ======

In [14]:
for idx, model in enumerate(models):
    weights = np.min(model.get_weights()[0]), np.max(model.get_weights()[0]), np.min(model.get_weights()[1]), np.max(model.get_weights()[1])
    print(model.layers[1].kernel_regularizer.l2, model.layers[1].kernel_initializer)
    print(weights)
    print(results[idx])
    print('-----------')

0.008 <keras.initializers.initializers_v2.GlorotNormal object at 0x0000023A96173CA0>
(-0.7339059, 0.83242553, -1.3700318, 1.2127553)
[0.37099191546440125, 0.9836100339889526, 0.9004328846931458, 0.9629629850387573, 0.9487603306770325]
-----------
0.007 <keras.initializers.initializers_v2.GlorotNormal object at 0x0000023A973B4130>
(-0.731272, 0.8421514, -1.5766361, 1.1370975)
[0.3565647602081299, 0.9828633666038513, 0.9134199023246765, 0.9504504799842834, 0.9487603306770325]
-----------
0.006 <keras.initializers.initializers_v2.GlorotNormal object at 0x0000023A95ECD730>
(-0.89648986, 0.7053443, -1.0143557, 1.4345479)
[0.3416169583797455, 0.9828518629074097, 0.9177489280700684, 0.9464285969734192, 0.9487603306770325]
-----------
0.008 <keras.initializers.initializers_v2.HeNormal object at 0x0000023A9B929A30>
(-0.7795853, 0.82940185, -1.4127246, 1.3651894)
[0.3692987263202667, 0.9840613603591919, 0.9090909361839294, 0.963302731513977, 0.9520661234855652]
-----------
0.007 <keras.initializ

In [331]:
# STARE
# for idx, model in enumerate(models):
#     weights = np.min(model.get_weights()[0]), np.max(model.get_weights()[0]), np.min(model.get_weights()[1]), np.max(model.get_weights()[1])
#     print(model.layers[1].kernel_regularizer.l2, model.layers[1].kernel_initializer)
#     print(weights)
#     print(results[idx])
#     print('-----------')

0.008 <keras.initializers.initializers_v2.GlorotNormal object at 0x000001A5D7BE7610>
(-0.850345, 0.8334865, -1.2260424, 1.6032988)
[0.368624746799469, 0.9454545378684998]
-----------
0.007 <keras.initializers.initializers_v2.GlorotNormal object at 0x000001A5D7BDA310>
(-0.7647453, 0.92581064, -1.3860332, 1.1354349)
[0.35585373640060425, 0.9438016414642334]
-----------
0.006 <keras.initializers.initializers_v2.GlorotNormal object at 0x000001A5DAE54400>
(-0.6860807, 0.85149384, -1.2082765, 1.5861523)
[0.3309980630874634, 0.9553719162940979]
-----------
0.008 <keras.initializers.initializers_v2.HeNormal object at 0x000001A5DAE8B4F0>
(-0.6180524, 0.74834824, -1.3511591, 1.3180828)
[0.3736017048358917, 0.9520661234855652]
-----------
0.007 <keras.initializers.initializers_v2.HeNormal object at 0x000001A5DC0BCD00>
(-0.8114171, 0.7657546, -1.3345479, 1.0654538)
[0.34865325689315796, 0.9570248126983643]
-----------
0.006 <keras.initializers.initializers_v2.HeNormal object at 0x000001A5DC241400>

In [333]:
models[4].save_weights('./checkpoints/my_checkpoint2')

In [334]:
models[4].save('./saved_model2')

INFO:tensorflow:Assets written to: ./saved_model2\assets


INFO:tensorflow:Assets written to: ./saved_model2\assets


In [299]:
models[1].summary()

Model: "sequential_113"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_113 (Flatten)       (None, 10)                0         
                                                                 
 dense_226 (Dense)           (None, 10)                100       
                                                                 
 custom_sigmoid (Activation)  (None, 10)               0         
                                                                 
 dense_227 (Dense)           (None, 1)                 10        
                                                                 
Total params: 110
Trainable params: 110
Non-trainable params: 0
_________________________________________________________________


In [321]:
tf.keras.utils.plot_model(models[1], to_file="my_model.png", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [314]:
from platform import python_version

In [315]:
python_version()

'3.9.13'

AttributeError: 'Sequential' object has no attribute 'result'